In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import imageio

# Load Dataset CIFAR-10

In [ ]:
# Charger le dataset CIFAR-10
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

# Normaliser les images d'entraînement et de test pour qu'elles soient entre -1 et 1
train_images = (train_images - 127.5) / 127.5
test_images = (test_images - 127.5) / 127.5

# Vérifier la forme des datasets pour s'assurer que la normalisation est correcte
print("Shape des images d'entraînement :", train_images.shape)
print("Valeur minimum des images d'entraînement :", train_images.min())
print("Valeur maximum des images d'entraînement :", train_images.max())

print("Shape des images de test :", test_images.shape)
print("Valeur minimum des images de test :", test_images.min())
print("Valeur maximum des images de test :", test_images.max())


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(10000).batch(256)

In [ ]:
# Fonction pour afficher un échantillon d'images
def display_sample_images(images, num_images=25):
    plt.figure(figsize=(10, 10))
    for i in range(num_images):
        plt.subplot(5, 5, i + 1)  # Grille 5x5
        plt.imshow((images[i] + 1) / 2)  # Re-normaliser entre 0 et 1 pour l'affichage
        plt.axis('off')  # Masquer les axes
    plt.show()

# Afficher un échantillon d'images d'entraînement
display_sample_images(train_images)

# Create GAN Fonctions

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense (8*8*256, use_bias=False, input_shape=(100,))) 
    model.add(layers. BatchNormalization())
    model.add(layers. LeakyReLU())
    model.add(layers. Reshape((8, 8, 256)))
    assert model.output_shape == (None, 8, 8, 256)  #test a condition, if true continue else error
    # None représente la taille du batch
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers. BatchNormalization())
    model.add(layers. LeakyReLU())
    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers. BatchNormalization())
    model.add(layers. LeakyReLU())
    model.add(layers. Conv2DTranspose(3, (5, 5), strides=(1, 1), padding='same', use_bias=False, activation= 'tanh')) 
    assert model.output_shape == (None, 32, 32, 3)
    return model

def make_discriminator_model():
    model = tf.keras. Sequential()
    model.add(layers. Conv2D (64, (5, 5), strides=(2, 2), padding='same', input_shape=[32, 32, 3]))
    model.add(layers. LeakyReLU())
    model.add(layers. Dropout (0.3))
    model.add(layers.Conv2D (128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers. Dropout (0.3))
    model.add(layers. Flatten())
    model.add(layers. Dense (1))
    return model

In [ ]:

# Définition de la fonction de perte pour l'entropie croisée binaire
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Fonction de perte pour le discriminateur
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)    # le modèle doit prédire 1    
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)   # le modèle doit prédire 0
    # Somme des deux pertes
    total_loss = real_loss + fake_loss
    return total_loss

# Fonction de perte pour le générateur
def generator_loss(fake_output):
    # Le générateur essaie de faire classer les fausses images comme réelles (1)
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Définition des optimiseurs pour les deux modèles
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

# Visualise an image generation without training

In [ ]:
# Initialiser les modèles générateur et discriminateur
generator = make_generator_model()
discriminator = make_discriminator_model()

# Générer une image avec le générateur non entraîné
noise = tf.random.normal([1, 100])  # Bruit d'entrée pour le générateur
generated_image = generator(noise, training=False)  # Générer une image

view_generated_image = (generated_image[0] + 1) / 2.0

# Afficher l'image générée
plt.imshow(view_generated_image)
plt.axis('off')  # Masquer les axes pour une meilleure visualisation
plt.show()
# Évaluer l'image générée avec le discriminateur non entraîné
decision = discriminator(generated_image, training=False)

# Afficher les résultats
print("Image générée (forme) :", generated_image.shape)
print("Sortie du discriminateur pour l'image générée :", decision.numpy())


# Create a Fonction to vizualise a grid of images every epoch

In [ ]:
# Fonction pour afficher les images générées en grille
def display_images(images, epoch=None):
    plt.figure(figsize=(4, 4))
    for i in range(images.shape[0]):
        plt.subplot(4, 4, i + 1)
        img = (images[i] + 1) / 2.0  # Redimensionner l'image pour être entre 0 et 1
        plt.imshow(img)
        plt.axis('off')

    if epoch is not None:
        plt.suptitle(f"Images générées à l'époque {epoch}")
    plt.show()

In [ ]:
# Définir la dimension du bruit
noise_dim = 100

# Créer un vecteur de bruit constant pour visualiser les progrès au cours des époques
# Générer 16 vecteurs de bruit pour obtenir 16 images
num_examples_to_generate = 16
fixed_seed = tf.random.normal([num_examples_to_generate, noise_dim])

generator = make_generator_model()

# Générer des images en utilisant le vecteur de bruit fixe
generated_images = generator(fixed_seed, training=False)

# Afficher les images générées
display_images(generated_images)


# Create Train fonction

`@tf.function` optimise et accélère l'exécution d'une fonction TensorFlow en la transformant en un graphe de calcul efficace, idéal pour des opérations intensives comme l'entraînement de réseaux de neurones.

`with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape`: creates two separate context managers for automatic differentiation in TensorFlow, allowing you to compute gradients for both the generator (gen_tape) and discriminator (disc_tape) models simultaneously during the training step.

In [ ]:
# Define the training step function
@tf.function
def train_step(images):
    # Generate random noise for the generator
    noise = tf.random.normal([images.shape[0], noise_dim])

    # Use a GradientTape to track the operations for automatic differentiation
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # Generate fake images from the noise
        generated_images = generator(noise, training=True)

        # Get the discriminator output for real and fake images
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        # Calculate the loss for the discriminator and generator
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    # Calculate the gradients for both models
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    # Apply the gradients to the optimizers
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    # Return the losses to monitor training
    return gen_loss, disc_loss

In [ ]:
noise_dim = 100
num_examples_to_generate = 16
fixed_seed = tf.random.normal([num_examples_to_generate, noise_dim])

def generate_and_save_images(model, epoch, test_input):
    # Générer des images à partir du bruit d'entrée
    generated_images = model(test_input, training=False)

    # Configurer la taille de la figure pour afficher les images
    plt.figure(figsize=(4, 4))
    for i in range(generated_images.shape[0]):
        plt.subplot(4, 4, i + 1)
        # Normaliser l'image pour qu'elle soit entre 0 et 1
        img = (generated_images[i] + 1) / 2.0
        plt.imshow(img)
        plt.axis('off')  # Masquer les axes

    # Enregistrer la figure sous forme d'image PNG
    plt.savefig(f'generated_images_epoch_{epoch}.png')  # Nom du fichier avec l'époque
    plt.close()  # Fermer la figure pour libérer de la mémoire

In [ ]:
# Fonction d'entraînement
def train(dataset, epochs):
    # Itérer sur le nombre d'époques
    for epoch in range(epochs):
        # Boucle sur chaque lot d'images dans le dataset
        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch)
        generate_and_save_images(generator, epoch + 1, fixed_seed)  # Générer des images après chaque époque

        # Afficher un message de progression
        print(f'Epoque {epoch + 1}, Perte du Générateur: {gen_loss.numpy()}, Perte du Discriminateur: {disc_loss.numpy()}')

        # Générer des images à partir d'un bruit fixe pour visualiser les progrès
        generated_images = generator(fixed_seed, training=False)  # Utiliser le vecteur de bruit fixe
        #display_images(generated_images, epoch + 1)  # Afficher les images générées

In [ ]:
# Exécuter l'entraînement
train(train_dataset, epochs=1000)

# Create a gif form generated images in epochs

In [ ]:
# Fonction pour afficher une image spécifique basée sur l'époque
def display_image(epoch):
    # Charger l'image générée à partir d'un fichier
    img = mpimg.imread(f'generated_images_epoch_{epoch}.png')
    
    # Afficher l'image
    plt.imshow(img)
    plt.axis('off')  # Masquer les axes
    plt.title(f'Image générée à l\'époque {epoch}')  # Titre de l'image
    plt.show()

# Exemple d'utilisation
display_image(epoch=5)


In [ ]:
import imageio.v2 as imageio  # Importer imageio v2
import os

# Fonction pour créer un GIF animé à partir des images sauvegardées
def create_gif_from_images(gif_name='progress.gif', num_epochs=10):
    with imageio.get_writer(gif_name, mode='I', duration=0.5) as writer:
        for epoch in range(1, num_epochs + 1):
            # Vérifier si l'image existe avant de l'ajouter au GIF
            image_path = f'/kaggle/working/generated_images_epoch_{epoch}.png'
            if os.path.exists(image_path):
                image = imageio.imread(image_path)  # Utiliser imageio.v2.imread
                writer.append_data(image)

# Exemple d'utilisation
create_gif_from_images(num_epochs=10)


In [ ]:
from IPython.display import HTML

# Chemin du GIF
gif_path = 'progress.gif'

# Afficher le GIF avec IPython
HTML(f'<img src="{gif_path}" alt="progress gif">')
